In [1]:
import copy
import os
import pickle
import subprocess

import httpx
from dotenv import load_dotenv
from google import genai
from google.genai import types as gemini_types
from langchain_google_genai import ChatGoogleGenerativeAI
from tqdm import tqdm

from eduly.models import Breakdown, TopicStoryboard

load_dotenv(dotenv_path="../../../.env", override=True)

aistudio_gemini_api_key = os.environ['GOOGLE_API_KEY']
print(aistudio_gemini_api_key[:3] + '...' + aistudio_gemini_api_key[-1:])
ai_studio_gemini_client = genai.Client(api_key=aistudio_gemini_api_key)


vertex_gemini_key = os.environ["VERTEX_AI_KEY"]
headers = {"Authorization": f"Basic {vertex_gemini_key}"}
base_url = os.environ["VERTEX_AI_BASE_URL"]

http_options=gemini_types.HttpOptions(
        headers=headers,
        api_version="",
        base_url=base_url,
        httpx_client=httpx.Client(verify=False)
    )


gemini_client = genai.Client(
    api_key="dummy_value_to_by_pass_validation",
    http_options=http_options
)

MODEL_NAME = "gemini-3-pro-preview"

langchain_client = ChatGoogleGenerativeAI(
    model=MODEL_NAME,
    temperature=1.0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
langchain_client.client = gemini_client



AIz...8


In [ ]:
# import copy
# import os
# import pickle
# import subprocess

# from dotenv import load_dotenv
# from langchain.chat_models import init_chat_model
# from tqdm import tqdm

# from eduly.models import Breakdown, TopicStoryboard

# load_dotenv(dotenv_path="../../../.env", override=True)

# BASE_URL = os.environ["VLLM_BASE_URL"]
# API_KEY = os.environ["VLLM_API_KEY"]
# MODEL_NAME = os.environ["VLLM_MODEL_NAME"]

# print(f"Base URL: {BASE_URL[:3]}...{BASE_URL[-1:]}")
# print(f"API Key: {API_KEY[:3]}...{API_KEY[-1:]}")
# print(f"Model Name: {MODEL_NAME}")

# langchain_client = init_chat_model(
#     model=MODEL_NAME,
#     model_provider="openai",
#     base_url=BASE_URL,
#     api_key=API_KEY,
# )


Base URL: htt...1
API Key: dap...8
Model Name: glm-4.7


In [3]:
langchain_client.invoke("Hello, world!")

AIMessage(content=[{'type': 'text', 'text': 'Hello! How can I help you today?', 'extras': {'signature': 'CuMHAY89a1/ViLPQ/fQ8XrkvUKQkXNs698IEPck5hUvOnF2qtpt78+Cd8i3WTokjkfSD7Ky3A82W6/zMLUNrMBqGoS+ObVS3FwTrJ6Rq7nHOTLnVFQ6yxfZ/ncNwg+AVyuMLVU6OiZYhG3QprXBTvIHoAOyMliv8YRmwlWjToyK3WlItB8NkS3aOrK/gSEw7ea9xVWJE8GZOp7kKXK85ZHCnFzpy3aFYS5x1Kn9BMoK1t/YhrLKbxsB36s/8l2GtklS1OiiiL2QWd/UdB9tRmU60+2dJ2OARJFiyohsbQhdryJhY+pyNGraDieZAsaqXCt2XrPmcwnbID6fxEw0PNUNDvFQy12qfetJzuB90vuDXAB8xW7//ldQdGxMQDg8yeEn/imyVaKwnZNUZEDKJOa0REGdXAAVtIAJ2/915lSjj78zwY+52uGPHpDkoXBEbkpJfkxsAiiHYcHTBRVBcpTIyB++wC8a5rrHsQLpW/7zCvuUhQ283GkODrAoZNhj4RrYlBQaCx28Wtk6RLL4Zf8SRJHyVRuJFzOnq/lXlG6v0BYWVdfV1icLSIgrUBpNQITvfBPbdQ6ZcIud1IIEmtue2b3T/fhnAmnV/GUpps0BTE/ez+9is3+XI1/B/5vQorrzjUyr4jt7ghrw4RX4Krz9sjtrd867rwfZQDwoLUPAl0SeHvs0pbUFzC5PSsc+ynYwN2xp0D5MH5fxkGglEvnxbLIGGIWage/OHpaLpiO/W7Q8tBwvIYfRvGSaTsZ/bAg6sxIGIb5ysHmUcy93u2UPPMwtMzZYkUWc8jmahWC/DQ6I/keOcvNmmybqiGKNoMcMFznVIEQNLvGxmtoN1x0FhrbDEQzNd4+Qo4HzeZCVABJJwotGjfSXsrZ8Jy11

In [7]:

from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend

manim_coding_agent_prompt = """You are an Expert Manim Animator creating detailed educational videos with access to documentation and a workspace.

## Your Goal
Complete the `workspace/scene.py` file with Manim code that implements all the requested animations in a single Scene class. Your animations should be **rich, detailed, and educational** — not simple or abstract.

## Workspace Structure
You have access to two folders:
- `manim_docs/` - **READ-ONLY** Manim documentation (tutorials, guides, API reference)
- `workspace/` - Contains `scene.py` file to complete (already has boilerplate code)

Your task is to **add a single Scene class with all animations below this boilerplate**. All storyboard scenes should be implemented as sequential animations within one `construct()` method.

## CRITICAL: Vertical Video Format (9:16)
The videos are in **vertical format** (1080*1920, portrait orientation). The scene measures **8 units in width and 14 units in height** (9:16 ratio). Keep this in mind:
- **Frame is tall and narrow** — you have much more vertical space than horizontal
- **Stack elements vertically** rather than spreading horizontally
- **Use more vertical space** — UP/DOWN positioning is preferred over LEFT/RIGHT
- **Text and objects should be sized appropriately** for the narrower width, use smaller fonts and objects if needed
- **Avoid wide layouts** — they will be cut off or cramped

## CRITICAL: Concept Caption Bar (Bottom Text Overlay)
**EVERY scene MUST have a "Concept Caption" text bar at the bottom of the screen** that highlights the key concept being shown:
- Create a semi-transparent dark rectangle at the bottom (spanning full width, ~1.5 units tall)
- Display the key concept/idea as white text on this bar (font_size 28-32)
- Update this caption text when the concept changes using Transform or FadeTransition
- Position: `DOWN * 6` to `DOWN * 6.5` (bottom area of the 14-unit tall frame)
- Example concepts: "Residual connections preserve gradient flow", "Matrix multiplication combines features", "Softmax normalizes attention weights"
- Keep captions concise but informative (1-2 sentences max)

## CRITICAL: Detailed, Non-Abstract Visualizations
Your animations must be **detailed and concrete**, NOT simple/abstract:
- **Use real data examples**: Show actual numbers, vectors with values, matrices with entries
- **Label everything**: Every component should have clear text labels
- **Show intermediate steps**: Don't skip from input to output — show the transformation process
- **Use color coding**: Different colors for different concepts (queries=blue, keys=gold, values=green, etc.)
- **Include annotations**: Add arrows, brackets, and explanatory text pointing to key parts
- **Show formulas alongside visuals**: When a computation happens, show the math equation next to it
- **Progressive reveal**: Build up complex diagrams step by step, not all at once
- **Concrete examples**: Instead of "a vector", show [0.3, 0.8, 0.1, 0.5] with indices labeled

## CRITICAL: Research Before Coding
You have documentation tools available and you MUST use them before writing any Manim code:
- **ALWAYS** look up the exact API for classes, methods, and parameters you plan to use
- **NEVER** guess or rely on memory—verify everything in the docs first
- **RESEARCH FIRST**, then write code based on what you find

## Your Tools

| Tool | Description |
|------|-------------|
| `ls` | List files in a directory with metadata (size, modified time) |
| `read_file` | Read file contents with line numbers, supports offset/limit for large files |
| `write_file` | Create new files |
| `edit_file` | Perform exact string replacements in files (with global replace mode) |
| `glob` | Find files matching patterns (e.g., `**/*.py`) |
| `grep` | Search file contents with multiple output modes (files only, content with context, or counts) |

## Documentation Structure (in `manim_docs/`)
- `tutorials/` - Getting started tutorials with code examples
  - `quickstart.md` - Basic Scene patterns, Circle, Square, Transform, .animate syntax
  - `building_blocks.md` - Mobjects, animations, scenes fundamentals
  - `output_and_config.md` - Rendering settings and CLI options
- `guides/` - In-depth how-to guides
  - `using_text.md` - Text, MarkupText, Tex, MathTex rendering
  - `configuration.md` - Config options and customization
  - `deep_dive.md` - Manim internals and advanced concepts
  - `add_voiceovers.md` - Adding audio narration
- `reference_index/` - Category indices (start here to find which module you need)
  - `animations.md` - Animation classes by category (creation, fading, transform, etc.)
  - `mobjects.md` - Mobject classes by category (geometry, text, graph, table, etc.)
  - `scenes.md` - Scene types (Scene, ThreeDScene, MovingCameraScene, etc.)
  - `cameras.md`, `configuration.md`, `utilities_misc.md`
- `reference/` - Detailed API docs for individual classes/functions (385 .md files)
  - Named as `manim.<module>.<class>.md` (e.g., `manim.animation.creation.Create.md`)
  - Contains parameters, methods, attributes, and usage examples
- `reference.md` - Full hierarchical module index (lists all classes organized by module)

## Required Workflow
1. **Read the scene file**: Use `read_file workspace/scene.py` to see the existing boilerplate
2. **Identify what you need**: What classes/methods will implement the storyboard animations?
3. **Search the docs**: Use `glob` or `grep` to find relevant documentation in `manim_docs/`
4. **Read the details**: Use `read_file` to get exact parameters, signatures, and examples
5. **Complete the scene**: Use `edit_file` to add your Scene class code to `workspace/scene.py`

## Key Manim Patterns (for reference)
- Scenes: Subclass `Scene`, implement `construct()`, use `self.play()` for animations
- Mobjects: Circle, Square, Text, MathTex, Arrow, VGroup, Matrix, Table, etc.
- Animations: Create, FadeIn, FadeOut, Transform, ReplacementTransform, Write, GrowArrow
- Positioning: `.move_to()`, `.next_to()`, `.shift()`, constants like UP, DOWN, LEFT, RIGHT
- Animate syntax: `mobject.animate.method()` to animate property changes

## Vertical Layout Tips
- Place titles at the top (UP * 5 or higher)
- Stack content vertically with consistent spacing
- Use `.scale()` to ensure objects fit within width=8 units
- Consider using `VGroup` with `.arrange(DOWN)` for vertical stacking
- **Reserve bottom 1.5-2 units for the concept caption bar**

## Detail & Richness Checklist
Before finalizing your code, verify:
- [ ] Every scene has a concept caption at the bottom
- [ ] All visual elements are labeled with text
- [ ] Numbers/values are shown where applicable (not just abstract shapes)
- [ ] Colors are meaningful and consistent across scenes
- [ ] Complex concepts are built up incrementally
- [ ] Mathematical formulas appear alongside their visual representations
- [ ] Transitions between scenes are smooth (FadeOut/FadeIn)

Remember: ALWAYS use your tools to research the documentation before writing code. Complete `workspace/scene.py` by adding your Scene class with all animations below the existing boilerplate."""


manim_animator_agent = create_deep_agent(
    model=langchain_client,
    system_prompt=manim_coding_agent_prompt,
    backend=FilesystemBackend(root_dir="./agent_workspace", virtual_mode=True),
)


def render_scene(agent_workspace_dir='./agent_workspace/workspace'):
    my_env = os.environ.copy()
    my_env["PATH"] = "/Library/TeX/texbin:" + os.environ.get("PATH", "")

    manim_result = subprocess.run(
        ['uv', 'run', 'manim', '-ql', 'scene.py'],
        capture_output=True,
        text=True,
        cwd=os.path.dirname(os.path.abspath(agent_workspace_dir)) + '/workspace',
        env=my_env
    )

    return manim_result

In [8]:
with open('./deepseek_mhc/deepseek_mhc_breakdown.pkl', 'rb') as f:
    breakdown = pickle.load(f)
breakdown = Breakdown.model_validate_json(breakdown.text)


In [9]:
for i in range(len(breakdown.topics)):
    print(i, breakdown.topics[i].name)

0 The Evolution of Residual Connections
1 Hyper-Connections (HC): Expanding the Residual Stream
2 The Stability Crisis: Signal Explosion in HC
3 mHC: The Manifold-Constrained Solution
4 The Sinkhorn-Knopp Algorithm: Enforcing Constraints
5 Breaking the Memory Wall: Infrastructure Optimization
6 Memory Management: Selective Recomputing
7 DualPipe: Overlapping Communication and Computation
8 Results: 27B Model Benchmarks
9 Modern Context: DeepSeek and the Future of Architecture


In [10]:
existing_videos = [int(x.split('_')[-1].replace('.mp4', '')) for x in os.listdir('./deepseek_mhc/') if x.endswith('.mp4')]
existing_videos

[]

In [11]:
import time

# ratelimit = False
ratelimit = True

In [12]:

for TOPIC_INDEX in tqdm(range(len(breakdown.topics))):

    if TOPIC_INDEX in existing_videos:
        print(f"Skipping topic {TOPIC_INDEX} because it already exists")
        continue


    with open(f'./deepseek_mhc/deepseek_mhc_gemini_3_flash_storyboard_{TOPIC_INDEX}.pkl', 'rb') as f:
        storyboard = pickle.load(f)

    storyboard = TopicStoryboard.model_validate_json(storyboard.text)

    # Get topic context for the agent
    current_topic = breakdown.topics[TOPIC_INDEX]
    total_topics = len(breakdown.topics)
    previous_topic = breakdown.topics[TOPIC_INDEX - 1] if TOPIC_INDEX > 0 else None
    next_topic = breakdown.topics[TOPIC_INDEX + 1] if TOPIC_INDEX < total_topics - 1 else None

    subprocess.run(["rm -rf ./workspace/*"], shell=True, capture_output=True, text=True, cwd=os.path.dirname(os.path.abspath("./agent_workspace/workspace/")))

    boilerplate_code = """# The videos are meant to be in vertical format (1080*1920, portrait orientation).
# Manually set the frame size, height, and width to ensure the scene is rendered correctly.
# Note: Now the scene measures 8 units in width and 14 units in height (9:16 ratio).

from manim import *

config.frame_size = [1080, 1920]
config.frame_height = 14
config.frame_width = 8
"""


    with open('./agent_workspace/workspace/scene.py', 'w') as f:
        f.write(boilerplate_code)


    storyboard_text = ""
    for i, scene in enumerate(storyboard.scenes):
        storyboard_text += f"## Scene {i+1}\n"
        storyboard_text += scene.to_text() + '\n'

    # Build series context
    series_context = f"""# Document Context
    **Document:** {breakdown.document_title}
    **Summary:** {breakdown.document_summary}

    # Series Navigation
    This is **Part {TOPIC_INDEX + 1} of {total_topics}** in the series on "{breakdown.document_title}".

    **All Topics in Series:**
    """
    for i, topic in enumerate(breakdown.topics):
        marker = "👉 " if i == TOPIC_INDEX else "   "
        series_context += f"{marker}{i + 1}. {topic.name}\n"

    # Previous/Next topic info
    if previous_topic:
        series_context += f"""
    **Previous Topic (Part {TOPIC_INDEX}):** {previous_topic.name}
    - Summary: {previous_topic.summary}
    """

    if next_topic:
        series_context += f"""
    **Next Topic (Part {TOPIC_INDEX + 2}):** {next_topic.name}
    - Summary: {next_topic.summary}
    """

    # Topic details from breakdown
    topic_context = f"""
    # Current Topic Details
    **Topic:** {current_topic.name}
    **Summary:** {current_topic.summary}

    **Full Explanation:**
    {current_topic.full_explanation}

    **Key Takeaways:**
    """
    for takeaway in current_topic.key_takeaways:
        topic_context += f"- {takeaway}\n"

    storyboard_prompt = f"""
    {series_context}
    {topic_context}

    ---

    # Storyboard: {storyboard.topic_name}

    You will implement this storyboard as a Manim animation. The visual descriptions specify what to render; the narration is for audio only (don't render narration text).

    ## Scene 0: Opening Frame
    Display a brief series header like "Part {TOPIC_INDEX + 1}/{total_topics}: {breakdown.document_title}" at the top (small, subtle), then a hook on what we will learn in this topic.
    **Concept Caption:** "Understanding {storyboard.topic_name.split(':')[0] if ':' in storyboard.topic_name else storyboard.topic_name}"

    {storyboard_text}

    ## Final Scene: Closing & Preview
    After the last storyboard scene, add a closing sequence:
    1. Fade out the current content
    2. Show a summary of key takeaways from this topic (use the Key Takeaways above)
    {f'3. Add a "Coming Next" preview: "Next: {next_topic.name}" with a brief teaser about what viewers will learn' if next_topic else ''}

    ---

    # Implementation Notes
    1. Add ONE Scene class with all animations in `construct()`
    2. Use smooth transitions (FadeOut/FadeIn/Transform) between scenes
    3. VERTICAL FORMAT: 8 units wide * 14 tall - stack elements vertically
    4. Complete `workspace/scene.py` below the existing boilerplate
    5. Use the document context and topic explanation to ensure accurate technical content
    6. The opening should briefly acknowledge the series context (part X of Y)
    7. The closing should tease the next topic to encourage continued viewing
    """

    agent_workspace_dir = './agent_workspace/workspace'
    print("running manim_animator_agent iter 0")
    result = manim_animator_agent.invoke({"messages": [{"role": "user", "content": storyboard_prompt}]})
    if ratelimit:
        print('ratelimit : sleeping for 60 seconds')
        time.sleep(60)
    manim_result = render_scene()
    try:
        success = len([x for x in os.listdir(os.path.dirname(os.path.abspath(agent_workspace_dir)) + '/workspace/media/videos/scene/1920p15/') if x.endswith('.mp4')]) > 0
    except FileNotFoundError:
        success = False

    iteration = 0

    while not success and iteration < 3:
        print(f"Iteration {iteration}")
        history = copy.deepcopy(result['messages'])
        history.append({"role": "assistant", "content": manim_result.stderr})
        print(f"Invoking manim_animator_agent with history of length {len(history)}")
        result = manim_animator_agent.invoke({"messages": history})
        if ratelimit:
            print('ratelimit : sleeping for 60 seconds')
            time.sleep(60)
        print(f"Completed iteration {iteration}, running render_scene()")
        manim_result = render_scene()

        try:
            success = len([x for x in os.listdir(os.path.dirname(os.path.abspath(agent_workspace_dir)) + '/workspace/media/videos/scene/1920p15/') if x.endswith('.mp4')]) > 0
        except FileNotFoundError:
            success = False
        iteration += 1

    if success:
        print("Successfully generated video")
        print("file: ", agent_workspace_dir + '/media/videos/scene/1920p15/' + [x for x in os.listdir(os.path.dirname(os.path.abspath(agent_workspace_dir)) + '/workspace/media/videos/scene/1920p15') if x.endswith('.mp4')][0])
    else:
        print("Failed to generate video at iteration", iteration)

    if success:
        video_path = agent_workspace_dir + '/media/videos/scene/1920p15/' + [x for x in os.listdir(os.path.dirname(os.path.abspath(agent_workspace_dir)) + '/workspace/media/videos/scene/1920p15') if x.endswith('.mp4')][0]
        subprocess.run(["cp", video_path, f"./deepseek_mhc/deepseek_mhc_animation_{TOPIC_INDEX}.mp4"])


  0%|          | 0/10 [00:00<?, ?it/s]

running manim_animator_agent iter 0
ratelimit : sleeping for 60 seconds
Iteration 0
Invoking manim_animator_agent with history of length 25
ratelimit : sleeping for 60 seconds
Completed iteration 0, running render_scene()
Iteration 1
Invoking manim_animator_agent with history of length 33
ratelimit : sleeping for 60 seconds
Completed iteration 1, running render_scene()


 10%|█         | 1/10 [07:00<1:03:07, 420.80s/it]

Successfully generated video
file:  ./agent_workspace/workspace/media/videos/scene/1920p15/EvolutionOfResiduals.mp4
running manim_animator_agent iter 0
ratelimit : sleeping for 60 seconds


 20%|██        | 2/10 [10:34<39:50, 298.76s/it]  

...

Successfully generated video
file:  ./agent_workspace/workspace/media/videos/scene/1920p15/PartNine.mp4
running manim_animator_agent iter 0
ratelimit : sleeping for 60 seconds
Iteration 0
Invoking manim_animator_agent with history of length 27


Gemini produced an empty response. Continuing with empty message
Feedback: None


ratelimit : sleeping for 60 seconds
Completed iteration 0, running render_scene()
Iteration 1
Invoking manim_animator_agent with history of length 29
ratelimit : sleeping for 60 seconds
Completed iteration 1, running render_scene()


100%|██████████| 10/10 [1:03:00<00:00, 378.00s/it]

Successfully generated video
file:  ./agent_workspace/workspace/media/videos/scene/1920p15/DeepSeekArchitecture.mp4
